In [16]:
print("Hello Speckle Tutorial Get Block Autocad!")

Hello Speckle Tutorial Get Block Autocad!


In [56]:
# Install update Specklepy 
!pip install --upgrade pip
!pip install --upgrade specklepy

In [17]:
# Import Library Specklepy
import specklepy
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account

In [18]:
# Get your account info
account = get_default_account()
print("Name of account: " + account.userInfo.name)
client = SpeckleClient(host="speckle.xyz")
client.authenticate_with_account(account)

Name of account: Hồ Văn Chương


In [25]:
# Get your stream sent from Autocad
# url = https://speckle.xyz/streams/088f1b9f0d/commits/11ad702ec5
streamId = "088f1b9f0d" 
stream = client.stream.get(streamId)
print("Name of stream: " + stream.name)

Name of stream: AutocadTest


In [33]:
# Get all commit from your stream 
# get list of commits
commits = client.commit.list(streamId)
print("Number of commits: " + str(len(commits)))
# Get specific commit
commitId = "11ad702ec5"
commit = client.commit.get(streamId, commitId)


Number of commits: 4


In [38]:
from specklepy.transports.server import ServerTransport
from specklepy.api import operations
transport = ServerTransport(client=client, stream_id=streamId)
res = operations.receive(commit.referencedObject, transport)
blocks = res["@Blocks"]

In [39]:
# Get all blocks from your stream
print("Number of blocks: " + str(len(blocks)))

Number of blocks: 10


In [50]:
# Create a dictionary with information of blocks
data = []
for block in blocks:
    dictBlocks = {}
    dictBlocks["Name"] = block.definition.name
    dictBlocks["X"] = block.transform.matrix[3]
    dictBlocks["Y"] = block.transform.matrix[7]
    dictBlocks["Z"] = block.transform.matrix[11]
    data.append(dictBlocks)
# Create a dataframe with information of blocks
import pandas as pd
df = pd.DataFrame(data)
df


,Name,X,Y,Z
0,No host family,6.679914,4.881626,0
1,No host family,6.679914,2.843446,0
2,No host family,4.751273,2.339459,0
3,No host family,4.736437,4.073766,0
4,No host family,3.037749,3.577191,0
5,No host family,2.034083,5.656165,0
6,No host family,1.258367,3.576898,0
7,No host family,1.666880,1.576117,0
8,No host family,5.687441,0.712342,0
9,No host family,4.666589,5.858176,0


In [51]:
# Save dataframe to csv file
df.to_csv("blocks.csv", index=False)
